In [1]:
from looti import cosmo_emulator as cem
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

Load trained emulators

In [2]:
Looti = cem.CosmoEmulator(external_info={'config_yaml': '../readfile_configs/read_input4cast.yaml'})

In [3]:
Looti.FileReader.__dict__.keys()

dict_keys(['folders_path', 'params_file', 'reference_folder', 'save_path', 'save_name', 'z_file_name', 'k_file_name', 'ells_file_name', 'data_file_names', 'data_types', 'training_redshifts', 'n_samples', 'n_training_samples', 'n_test_samples', 'csv_zip', 'fileformat', 'n_folders', 'folders'])

In [4]:
Looti.create_dataframes()

Creating data frame for :  Plin
 Reference k-dataframe created 
Looping over 600 folders
Maximum number of n_samples reached
Time needed for Dataframe creation 48.8731
Number of parameters varying: 8
Parameters: ['w0_fld', 'wa_fld', 'Omega_m', 'Omega_b', 'h', 'n_s', 'A_s', 'sigma8']
Number of samples in dataset: 600
Dataframe saved to: ../training_data/class_Asw0wa_DP_cmb_2K_Plin.csv.zip
Reference dataframe saved to: ../training_data/class_Asw0wa_DP_cmb_2K_Plin_ref.csv.zip
------------------------------------------
Creating data frame for :  Pnonlin
 Reference k-dataframe created 
Looping over 600 folders
Maximum number of n_samples reached
Time needed for Dataframe creation 52.5827
Number of parameters varying: 8
Parameters: ['w0_fld', 'wa_fld', 'Omega_m', 'Omega_b', 'h', 'n_s', 'A_s', 'sigma8']
Number of samples in dataset: 600
Dataframe saved to: ../training_data/class_Asw0wa_DP_cmb_2K_Pnonlin.csv.zip
Reference dataframe saved to: ../training_data/class_Asw0wa_DP_cmb_2K_Pnonlin_ref.

FileNotFoundError: ../../raw_data/class_Asw0wa_DP_cmb_2K/0000/TE_cls.txt not found.

Set all parameters for training

In [ ]:
# path and file name of the training data set (pandas dataframe)
data_path = '../training_data/class_Asw0wa_DP_hypel_z0/'
file_name = 'Plin'

# type of observable to be emulated
cosmo_quantity = 'Plin'

# number of varying parameters in data set
n_params = 7

# set size of training and test data set
n_train = 575
n_test = 10

# choose redshifts
redshifts = [0]

# choose if during training the logarithm of the grid and spectrum should be used
features_to_Log = False
observable_to_Log = False

# choose number of PCA components
ncomp = 20

# option to have a seperate Gaussian process for each PCA component
mult_gp = True

Initialize Emulator

In [ ]:
LootiEmu_pk = cem.CosmoEmulator()

Read the pandas data frame, perform seperation into training and test sets, and normalize data

In [ ]:
LootiEmu_pk.read_data(cosmo_quantity=cosmo_quantity, 
                      data_path=data_path, 
                      file_name=file_name, 
                      n_params=n_params, 
                      n_train=n_train, 
                      n_test=n_test,
                      redshifts=redshifts,
                      features_to_Log=features_to_Log,
                      observable_to_Log=observable_to_Log)

Training

In [ ]:
LootiEmu_pk.create_emulator(cosmo_quantity=cosmo_quantity, n_params=n_params, mult_gp=mult_gp, ncomp=ncomp)

Access the data and the trained emulator

In [ ]:
# emulation_data_pk = LootiEmu_pk.data[cosmo_quantity]
intobj_pk = LootiEmu_pk.emu_objs[cosmo_quantity]

Check convergence by evaluating the emulator on the training dataset

In [ ]:
# input parameters of training dataset
xtrain = intobj_pk.trainspace_normed

# target values of PCA components for the training dataset
truth_normed_pcas_train = intobj_pk.representation

if mult_gp == True:
    # get predicted values of PCA components and uncertainty of the prediction from each GP
    pred_temp = []
    std_temp = []
    for comp, gp in enumerate(intobj_pk.gp_dict.values()):
        predictioncomp_train_temp, std_pca_train_temp = gp.predict(xtrain, return_std=True)
        pred_temp.append(list(predictioncomp_train_temp))
        std_temp.append(list(std_pca_train_temp))

    # predicted values of PCA components for the training dataset
    predictioncomp_train = np.array(pred_temp).T

    # standard deviations of all predictions for the training dataset
    std_pca_train = np.array(std_temp).T
    
else:
    # get prediction of values for all PCA component from GP
    predictioncomp_train, std_pca_train = intobj_pk.gp_regressor.predict(xtrain, return_std=True)

Plot the residuals (target minus predicted values of PCA components) for a small subset of the training dataset

In [ ]:
# looking at first 10 samples from training dataset
n_plot_train = 10

In [ ]:
pca_grid = np.linspace(1, n_plot_train, n_plot_train)
fig, axs = plt.subplots(nrows=int(np.ceil(ncomp/2)), ncols=2)

fig.suptitle('Residuals and Uncertainty of GP Predictions on Training Data Set')

for comp in range(ncomp):
    axs.ravel()[comp].errorbar(pca_grid, 
                 truth_normed_pcas_train[:n_plot_train][:,comp]-predictioncomp_train[:n_plot_train][:,comp], 
                 yerr=std_pca_train[:n_plot_train][:,comp], 
                 linestyle='', marker='.', color='cornflowerblue')
    axs.ravel()[comp].hlines(0, 1, n_plot_train, color='firebrick', alpha=0.3)

    axs.ravel()[comp].set_ylabel("Comp %i" %(comp+1))
    axs.ravel()[comp].set_xticks(list(range(1,n_plot_train+1)))
    axs.ravel()[comp].set_xticklabels([])

for i in range(2):
    axs[-1, i].set_xlabel('Train data index')
    axs[-1, i].set_xticks(list(range(1, n_plot_train+1)))
    axs[-1, i].set_xticklabels(list(range(1, n_plot_train+1)))

plt.tight_layout()
plt.show()

Evaluate the emulator on the test dataset

In [ ]:
xtest = (emulation_data_pk.test_samples - intobj_pk.trainspace_mean) / intobj_pk.trainspace_std

if mult_gp == True:
    pred_temp = []
    std_temp = []
    for comp, gp in enumerate(intobj_pk.gp_dict.values()):
        predictioncomp_test_temp, std_pca_test_temp = gp.predict(xtest, return_std=True)
        pred_temp.append(list(predictioncomp_test_temp))
        std_temp.append(list(std_pca_test_temp))
        
    predictioncomp_test = np.array(pred_temp).T
    std_pca_test = np.array(std_temp).T
    
else:
    predictioncomp_test, std_pca_test = intobj_pk.gp_regressor.predict(xtest, return_std=True)

Perform PCA on the test data to get target values for all PCA components

In [ ]:
nparam = emulation_data_pk.num_parameters + int(emulation_data_pk.multiple_z)
if emulation_data_pk.multiple_z:
    params = ['z']
else:
    params = []
params += list(emulation_data_pk.paramnames_dict.values())

truth_normed_pcas_list = []
for ii in range(emulation_data_pk.test_samples.shape[0]):
    indexvalues = emulation_data_pk.test_samples[ii]
    if emulation_data_pk.multiple_z:
        index_list = emulation_data_pk.data_type, indexvalues[0], params[1], indexvalues[1], params[2], indexvalues[2], params[3], indexvalues[3], params[4], indexvalues[4], params[5], indexvalues[5]
    else:
        index_list = emulation_data_pk.data_type, 0.0, params[0], indexvalues[0], params[1], indexvalues[1], params[2], indexvalues[2], params[3], indexvalues[3], params[4], indexvalues[4], params[5], indexvalues[5]

    truth_spectrum_test = emulation_data_pk.df_ext.loc[index_list].values.flatten()
    ref_spectrum = emulation_data_pk.df_ref.loc[emulation_data_pk.data_type, 0.0].values.flatten()

    truth_spectrum_normed_test = ((truth_spectrum_test/ref_spectrum) - emulation_data_pk.binwise_mean) / emulation_data_pk.binwise_std
    truth_pca_test_raw = intobj_pk.pca.transform([truth_spectrum_normed_test]).flatten()
    truth_pca_test = (truth_pca_test_raw - intobj_pk.matPCA_mean) / intobj_pk.matPCA_std
    truth_normed_pcas_list.append(truth_pca_test)

truth_normed_pcas = np.array(truth_normed_pcas_list)

Plot the residuals (target minus predicted values of PCA components) for the test dataset

In [ ]:
n_test = len(emulation_data_pk.test_samples)
pca_grid = np.linspace(1, n_test, n_test)
fig, axs = plt.subplots(nrows=int(np.ceil(ncomp/2)), ncols=2)

fig.suptitle('Residuals and Uncertainty of GP Predictions on Test Data Set')

for comp in range(ncomp):
    axs.ravel()[comp].errorbar(pca_grid, 
                 truth_normed_pcas[:,comp]-predictioncomp_test[:,comp], 
                 yerr=std_pca_test[:,comp], 
                 linestyle='', marker='.', color='cornflowerblue')
    axs.ravel()[comp].hlines(0, 1, n_test, color='firebrick', alpha=0.3)

    axs.ravel()[comp].set_ylabel("Comp %i" %(comp+1))
    axs.ravel()[comp].set_xticks(list(range(1,n_test+1)))
    axs.ravel()[comp].set_xticklabels([])

for i in range(2):
    axs[-1, i].set_xlabel('Test data index')
    axs[-1, i].set_xticks(list(range(1, n_test+1)))
    axs[-1, i].set_xticklabels(list(range(1, n_test+1)))

plt.tight_layout()
plt.show()

Plot the predicted vs. true spectra, and the residuals

In [ ]:
LootiEmu_pk.get_params(cosmo_quantity=cosmo_quantity)

In [ ]:
emulation_data_pk.df_ext

In [ ]:
colors = plt.cm.coolwarm(np.linspace(0, 1, len(emulation_data_pk.test_samples)))
fig, ax =plt.subplots(3, figsize=(7, 6))
fig.set_tight_layout(tight=True)

# get a list of the varying parameters
params_varying = list(LootiEmu_pk.get_params(cosmo_quantity=cosmo_quantity).keys())

# get true target spectra of the test dataset
test_indices = emulation_data_pk.test_splitdict[0]
truth_spectrum = emulation_data_pk.df_ext.loc[cosmo_quantity].values[test_indices]

for plot_index, color in enumerate(colors):
    # create input dictionary with values of test sample for each parameter
    input_values = emulation_data_pk.test_samples[plot_index]
    input_dict_pk = dict()
    for param, value in zip(params_varying, input_values):
        input_dict_pk[param] = value

    # get the grid and the predicted spectrum for the given input
    grid_temp, pk_test = LootiEmu_pk.get_prediction(cosmo_quantity=cosmo_quantity, input_dict=input_dict_pk)

    # transform grid in case it is given logarithmically
    if features_to_Log==True:
        grid = np.power(10, grid_temp)
    else:
        grid = grid_temp

    # get the target spectra for current index in test dataset
    pk_truth = truth_spectrum[plot_index]

    # upper plot: target and predicted spectrum 
    ax[0].loglog(grid, pk_truth, c='cornflowerblue', label='truth')
    ax[0].loglog(grid, pk_test, c='firebrick', linestyle='--', label='prediction')

    # middle plot: relative residuals
    residuals = 1 - pk_test / truth_spectrum[plot_index]
    ax[1].semilogx(grid, residuals, color=color)

    # lower plot: absolute residuals
    residuals = pk_test - truth_spectrum[plot_index]
    ax[2].semilogx(grid, residuals, color=color)

# set lables and title
ax[0].set_title('Prediction and Residuals for %i test samples' %(len(emulation_data_pk.test_samples)))
ax[0].set_ylabel('Spectra')
ax[0].set_xticklabels([])
ax[1].set_ylabel('Relative Residuals')
ax[1].set_xticklabels([])
ax[2].set_xlabel(r'$k$')
ax[2].set_ylabel('Residuals')

plt.tight_layout()
plt.show()

Chose path and file name to save trained emulator and dataset

In [ ]:
save_path = '../emulators/'
save_name = cosmo_quantity

Save emulator and dataset

In [ ]:
pickle.dump(emulation_data_pk, open(save_path+save_name+'test_data.sav', 'wb'))
pickle.dump(intobj_pk, open(save_path+save_name+'test.sav', 'wb'))